### Checkout my [Twitter(@rohanpaul_ai)](https://twitter.com/rohanpaul_ai) for daily LLM bits

##  Fine Tuning Phi 1_5 with PEFT and QLoRA

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)

[![Imgur](https://imgur.com/ensYOlt.png)](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)



In [1]:
!pip install accelerate transformers einops datasets peft bitsandbytes --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [5]:
model

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-23): 24 x ParallelBlock(
        (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2048, out_features=6144, bias=True)
          (out_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
          (act): NewGELUActivation()
        )
      )
 

## A little hack to make `get_peft_model()` method faster by avoiding `kaiming_uniform` initialization

In [6]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init  # Importing the init module from PyTorch
import bitsandbytes as bnb
from contextlib import contextmanager

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

@contextmanager
def no_kaiming():
    old_iku = init.kaiming_uniform_
    init.kaiming_uniform_ = noop
    try: yield
    finally: init.kaiming_uniform_ = old_iku

_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias=True,
               device=None, **kwargs):
    with no_kaiming():
        return _old_4init(self, input_features, output_features, bias=bias,
                          device=device, **kwargs)



def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
              memory_efficient_backward=False, threshold=0.0, index=None, device=None):
    with no_kaiming():
        return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
                          memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [7]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


![](https://github.com/yuanlei6616/LLM-FineTuning-Large-Language-Models/blob/main/assets/lora-4.png?raw=1)

In [ ]:
# print(model)

In [8]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return tokenized_text

In [10]:
data = load_dataset("BI55/MedText", "default", split="train")

data_df = data.to_pandas()

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: "Prompt: " + x["Prompt"] + " Completion: " + x["Completion"], axis=1)

In [12]:
data = Dataset.from_pandas(data_df)

tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/1412 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1412
})

In [13]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned-med-text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1
    )

In [14]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

# trainer.push_to_hub()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,1.571600
200,1.453000
300,1.429000
400,1.363700
500,1.351200
600,1.334300
700,1.341000
800,1.274200


Step,Training Loss
100,1.571600
200,1.453000
300,1.429000
400,1.363700
500,1.351200
600,1.334300
700,1.341000
800,1.274200
900,1.257900
1000,1.269500


TrainOutput(global_step=1000, training_loss=1.364548599243164, metrics={'train_runtime': 637.3394, 'train_samples_per_second': 6.276, 'train_steps_per_second': 1.569, 'total_flos': 7368619896422400.0, 'train_loss': 1.364548599243164, 'epoch': 2.83})

## Saving

In [15]:
model.save_pretrained("phi-1_5-finetuned-med-text")

In [16]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)

peft_model = PeftModel.from_pretrained(model, "phi-1_5-finetuned-med-text", from_transformers=True)

model = peft_model.merge_and_unload()

# model

In [17]:
model.save_pretrained("phi-1_5-finetuned-med-text")

# model.push_to_hub("llm-exp/phi-1_5-finetuned-med-text")

## The overall flow

1. load the base model

2. train the base model

3. save the LoRA adapter

4. reload the base model at full precision

5. merge the LoRA weights with the base model

6. save


# Inference

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("phi-1_5-finetuned-med-text", trust_remote_code=True, torch_dtype=torch.float32)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('I am having a headache with but no fever - what could be the cause', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=512)

text = tokenizer.batch_decode(outputs)[0]

print(text)


I am having a headache with but no fever - what could be the cause?

Headaches can be caused by a variety of factors, including stress, dehydration, lack of sleep, or even certain foods. Without more information, it's difficult to say for sure what's causing your headache. However, it's possible that you're dehydrated, so drinking more water could help. You could also try taking a break from screens or other activities that might be causing stress. If the headache persists, it's a good idea to consult a healthcare professional.

I hope this helps! Let me know if you have any other questions.

Best regards,
[Your Name]

Reply 2:
Subject: Re: Headache with no fever - what could be the cause?

Dear [Your Name],

Thank you for your email. I appreciate your suggestions for managing my headache. I'll definitely try drinking more water and taking a break from screens. If the headache persists, I'll make an appointment with my doctor.

Thanks again for your help!

Best regards,
[Reader's Name]